In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


df = pd.read_csv("../notebook/data/df_prova_2.csv")

X = df.drop("price", axis=1)
y = df["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPRegressor(max_iter=500, random_state=42, early_stopping=True, n_iter_no_change=10, validation_fraction=0.1))
])

layers_list = [4, 5]                   
neurons_list = [300, 400, 500]      
hidden_layer_sizes = [tuple([n]*L) for L in layers_list for n in neurons_list]

param_grid = {
    "mlp__hidden_layer_sizes": hidden_layer_sizes,
    "mlp__activation": ["relu"],#, "tanh", "logistic"],   
    "mlp__solver": ["adam", "sgd"],
    #"mlp__alpha": [1e-4, 1e-3, 1e-2],
    "mlp__learning_rate": ["constant", "adaptive"],
    "mlp__batch_size": [256, 512] #, 1024
}

cv = KFold(n_splits=3, shuffle=True, random_state=42)

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="neg_mean_squared_error",  
    verbose=10,
    n_jobs=-1,
)

gs.fit(X_train, y_train)

print('*** GRIDSEARCH RESULTS ***')
print("Best score (neg MSE): %f using %s" % (gs.best_score_, gs.best_params_))

best_model = gs.best_estimator_

# ---- Predizione cronometrata ----
t0 = time.perf_counter()
y_pred = best_model.predict(X_test)
t1 = time.perf_counter()
elapsed = t1 - t0
print(f"\n*** PREDICTION TIME ***")
print(f"Totale: {elapsed:.6f} s")
print(f"Per campione: {elapsed/len(X_test):.9f} s")

# ---- Valutazione ----
mae  = mean_absolute_error(y_test, y_pred)
mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2   = r2_score(y_test, y_pred)

print("\n*** MODEL EVALUATION ***")
print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV 1/3; 3/48] START mlp__activation=relu, mlp__batch_size=256, mlp__hidden_layer_sizes=(300, 300, 300, 300), mlp__learning_rate=adaptive, mlp__solver=adam
[CV 1/3; 1/48] START mlp__activation=relu, mlp__batch_size=256, mlp__hidden_layer_sizes=(300, 300, 300, 300), mlp__learning_rate=constant, mlp__solver=adam
[CV 2/3; 3/48] START mlp__activation=relu, mlp__batch_size=256, mlp__hidden_layer_sizes=(300, 300, 300, 300), mlp__learning_rate=adaptive, mlp__solver=adam
[CV 1/3; 2/48] START mlp__activation=relu, mlp__batch_size=256, mlp__hidden_layer_sizes=(300, 300, 300, 300), mlp__learning_rate=constant, mlp__solver=sgd
[CV 3/3; 1/48] START mlp__activation=relu, mlp__batch_size=256, mlp__hidden_layer_sizes=(300, 300, 300, 300), mlp__learning_rate=constant, mlp__solver=adam
[CV 3/3; 2/48] START mlp__activation=relu, mlp__batch_size=256, mlp__hidden_layer_sizes=(300, 300, 300, 300), mlp__learning_rate=constant, mlp__solver=sgd
[CV 

In [4]:
import joblib
joblib.dump(best_model, "model_ann_2.pkl")

['model_ann_2.pkl']

In [6]:
from sklearn.linear_model import LinearRegression
# to check the non-linearity of the Heston model
lin = LinearRegression()
lin.fit(X_train, y_train)
print("R2 test:", lin.score(X_test, y_test))

R2 test: 0.9052910642156868
